<a href="https://colab.research.google.com/github/vi21g/finetuning-gpt/blob/main/%D0%A2%D1%80%D0%B0%D0%BD%D1%81%D1%84%D0%BE%D1%80%D0%BC%D0%B5%D1%80%D1%8B_OpenAI_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune трансформера, обучаем писать заголовки

Для дообучения требуется датасет в формате со специальными токенами:

[TEXT] текст [TITLE] заголовок

Чтобы при инференсе подать:

[TEXT] текст [TITLE]

Модель должна будет сгенерировать заголовок

## инсталл

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.6 MB/s eta 0:00:00


## импорт

In [ ]:
from datasets import Dataset

In [ ]:
import kagglehub
import os

In [ ]:
import pandas as pd

In [ ]:
import re

## вспомогательные функции

In [ ]:
def remove_first_sentence(text):
    """Удаляет первое предложение из заданного текста.

    Args:
        text: Входная текстовая строка.

    Returns:
        Текст с удалённым первым предложением или исходный текст, если граница предложения не найдена.
    """
    match = re.search(r"[.!?](?:\s|$)", text)  # Найти границу первого предложения
    if match:
        return text[match.end():]  # Вернуть текст после первого предложения
    else:
        return text  # Вернуть исходный текст, если граница предложения не найдена

## загрузка модели

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

# Для большей наглядности будем работать с русскоязычной версией GPT от Сбера.
# Ниже находятся команды для загрузки и инициализации модели и токенизатора.
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(DEVICE)

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

## датасет

In [ ]:
path = kagglehub.dataset_download("vfomenko/russian-news-2020")

print("Path to dataset files:", path)

100%|██████████| 19.9M/19.9M [00:02<00:00, 9.29MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/vfomenko/russian-news-2020/versions/1


In [ ]:
os.listdir(path)

['news.csv']

In [ ]:
df = pd.read_csv(f'{path}/news.csv')

In [ ]:
df

,source,title,text,publication_date,rubric,subrubric,tags
0,lenta.ru,Синий богатырь,В 1930-е годы Советский Союз охватила лихорадк...,2020-08-30T00:01:00+03:00,Экономика,Госэкономика,NaN
1,lenta.ru,Загитова согласилась вести «Ледниковый период»,Олимпийская чемпионка по фигурному катанию Ал...,2020-08-31T20:04:00+03:00,Спорт,Зимние виды,NaN
2,lenta.ru,Объяснена опасность однообразного питания,Российский врач-диетолог Римма Мойсенко объясн...,2020-08-31T20:07:00+03:00,Из жизни,Еда,NaN
3,lenta.ru,«Предохраняться? А зачем?»,В 2019 году телеканал «Ю» запустил адаптацию з...,2020-08-30T00:04:00+03:00,Интернет и СМИ,ТВ и радио,NaN
4,lenta.ru,Ефремов систематически употреблял наркотики,Актер Михаил Ефремов систематически употребл...,2020-08-31T18:27:00+03:00,Культура,Кино,NaN
...,...,...,...,...,...,...,...
21668,tjournal.ru,\n Россия прекратил...,\n\n \n \n \n \n \n...,1578056674,NaN,NaN,NaN
21669,tjournal.ru,\n Во Владивостоке ...,\n\n \n \n \n \n \n...,1577866951,NaN,NaN,NaN
21670,tjournal.ru,\n Дым от австралий...,\n\n \n \n \n \n \n...,1577864124,NaN,NaN,NaN
21671,tjournal.ru,\n Около 200 жителе...,\n\n \n \n \n \n \n...,1577894168,NaN,NaN,NaN


In [ ]:
df['source'].value_counts()

,count
source,
ria.ru,14755
lenta.ru,4510
meduza.io,1538
tjournal.ru,870


### посмотрим примеры текста

In [ ]:
df[df['source'] == 'ria.ru'].sample(5)

,source,title,text,publication_date,rubric,subrubric,tags
9850,ria.ru,Названы новые тенденции в пластической хирургии,"МОСКВА, 30 июн — РИА Новости. Седьмой размер ...",02:50 30.06.2020,NaN,NaN,"Здоровье, Общество"
10972,ria.ru,Киностудиям Голливуда разрешили возобновить съ...,"МОСКВА, 11 июн - РИА Новости. Власти округа ...",02:11 11.06.2020,NaN,NaN,"Культура, Коронавирус COVID-19, Голливуд, Лос-..."
8085,ria.ru,В Литве число заразившихся коронавирусом вырос...,"МОСКВА, 30 июл - РИА Новости. Число выявленны...",11:29 30.07.2020,NaN,NaN,"Распространение нового коронавируса, Коронавир..."
9018,ria.ru,"Лолиту собирают по частям в клипе на песню ""Ки...","МОСКВА, 14 июл — РИА Новости. Певица Лолита М...",01:33 14.07.2020,NaN,NaN,"Культура, Музыка, Новости культуры, Александр ..."
7934,ria.ru,В России работающим пенсионерам проведут перер...,"МОСКВА, 1 авг - РИА Новости. Традиционную кор...",00:37 01.08.2020,NaN,NaN,Общество


In [ ]:
df[df['source'] == 'lenta.ru'].sample(5)

,source,title,text,publication_date,rubric,subrubric,tags
2367,lenta.ru,В Москве умер 41 пациент с коронавирусом,"В Москве умер 41 пациент с коронавирусом, сооб...",2020-04-24T21:29:00+03:00,Россия,Общество,NaN
4291,lenta.ru,Три футболиста «Манчестер Сити» попали в сборн...,"Футболисты «Манчестер Сити» Рияд Марез, Кевин ...",2020-01-13T20:58:00+03:00,Спорт,Английский футбол,NaN
595,lenta.ru,Поезд сошел с рельсов и загорелся на мосту в США,Грузовой поезд сошел с рельсов и загорелся на ...,2020-07-29T20:39:00+03:00,Мир,Происшествия,NaN
1815,lenta.ru,Появились подробности о задержанных участниках...,Задержан четвертый предполагаемый участник пер...,2020-05-24T21:28:00+03:00,Силовые структуры,Криминал,NaN
349,lenta.ru,В Бурятии ликвидируют пять незаконных свалок,В Республике Бурятия до 2024 года планируют ли...,2020-08-12T20:08:26+03:00,Нацпроекты,Общество,NaN


In [ ]:
df[df['source'] == 'meduza.io'].sample(5)

,source,title,text,publication_date,rubric,subrubric,tags
5420,meduza.io,Три новых случая заболевания коронавирусной ин...,Три новых случая заболевания коронавирусной ин...,"11:57, 9 марта 2020",NaN,NaN,NaN
4826,meduza.io,"Умер Мурис — один из двух котов, живших в Рижс...","Умер кот Мурис — тот самый, что много лет жил ...","14:08, 26 мая 2020",NaN,NaN,NaN
5912,meduza.io,Первый ученый Средиземья,16 января в Провансе в возрасте 95 лет умер Кр...,"15:48, 17 января 2020",NaN,NaN,NaN
5940,meduza.io,«Выборы в Госдуму приобретают смысл»,15 января в своем послании Федеральному собран...,"16:45, 15 января 2020",NaN,NaN,NaN
5686,meduza.io,"МВД возбудило уголовное дело из-за пранкера, р...",Полиция возбудила уголовное дело о хулиганств...,"11:29, 10 февраля 2020",NaN,NaN,NaN


In [ ]:
df[df['source'] == 'tjournal.ru'].sample(5)

,source,title,text,publication_date,rubric,subrubric,tags
21398,tjournal.ru,\n «Медуза» опублик...,\n\n \n \n \n \n \n...,1583262970,NaN,NaN,NaN
21244,tjournal.ru,\n Достигнут новый ...,\n\n \n \n \n \n \n...,1585831845,NaN,NaN,NaN
21493,tjournal.ru,"\n Ну что, ждем пер...",\n\n \n \n \n \n \n...,1581443367,NaN,NaN,NaN
20805,tjournal.ru,\n В Черногории про...,\n\n \n \n \n \n \n...,1598821788,NaN,NaN,NaN
21083,tjournal.ru,\n Закупленные Моск...,\n\n \n \n \n \n \n...,1588771937,NaN,NaN,NaN


### немного обработки

In [ ]:
# удаляем из датафрейма source == tjournal.ru
df = df[df['source'] != 'tjournal.ru']

In [ ]:
df['source'].value_counts()

,count
source,
ria.ru,14755
lenta.ru,4510
meduza.io,1538


In [ ]:
# удаляем первое предложение из текста для новостей с ria.ru - проверка
df[df['source'] == 'ria.ru'].sample(5)['text'].apply(remove_first_sentence)

,text
16537,Банк Goldman Sachs на фоне паники на нефтян...
7166,Российские ученые прокомментировали в беседе ...
8465,Власти города Ситка на Аляске еще не предложи...
19054,Полиция в столице Черногории в ночь на воск...
19523,День Brexit – 31 января – будет ознаменован...


In [ ]:
# удаляем первое предложение из текста для новостей с ria.ru
df.loc[df['source'] == 'ria.ru', 'text'] = df.loc[df['source'] == 'ria.ru', 'text'].apply(remove_first_sentence)

<ipython-input-20-4d727de29906>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['source'] == 'ria.ru', 'text'] = df.loc[df['source'] == 'ria.ru', 'text'].apply(remove_first_sentence)


In [ ]:
df.sample(5)

,source,title,text,publication_date,rubric,subrubric,tags
581,lenta.ru,Для сибирского синхротрона подготовят инженерн...,В Новосибирском государственном техническом ун...,2020-07-30T19:06:53+03:00,Нацпроекты,Наука,NaN
18479,ria.ru,"Эксперты рассказали, что россияне считают ключ...","Хорошие отношения с руководством, а также тру...",03:14 05.02.2020 (обновлено: 19:31 03.03.2020),NaN,NaN,"Россия, Общество"
19725,ria.ru,Помпео и Нетаньяху обсудили по телефону развит...,Госсекретарь США Майк Помпео и премьер-мини...,03:17 15.01.2020,NaN,NaN,"Майк Помпео, Государственный департамент США, ..."
19046,ria.ru,"Девятая годовщина ""революции 25 января"" в Егип...","Девятая годовщина ""революции 25 января"" в Еги...",01:00 26.01.2020,NaN,NaN,"Египет, В мире"
17579,ria.ru,"""Мулан"" первым из диснеевских ремейков получил...","Экранизация ""Мулан"" стала первым диснеевским р...",01:09 20.02.2020,NaN,NaN,"Культура, фильмы 2020, Кино и сериалы, Новости..."


## подготовка данных для файнтюнинга

Для дообучения переведём текст в формат со специальными токенами:

[TEXT] text [TITLE] title

Чтобы при инференсе подать:

[TEXT] text [TITLE]

Модель должна будет сгенерировать заголовок

In [ ]:
formatted_data = []
for _, row in df.iterrows():
    formatted_text = f"[TEXT]{row['text']}[TITLE]{row['title']}"  # Формат с токенами
    formatted_data.append({"text": formatted_text})

In [ ]:
# Конвертация в формат HuggingFace Dataset
dataset = Dataset.from_pandas(pd.DataFrame(formatted_data))

In [ ]:
# Добавляем новые токены
special_tokens = ["[TEXT]", "[TITLE]"]
tokenizer.add_special_tokens({"additional_special_tokens": special_tokens})

# Обновляем размер эмбеддингов модели
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50259, 1024)

In [ ]:
def tokenize_function(examples):
    # Токенизируем текст
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,  # Максимальная длина контекста
        padding="max_length"  # Добавляем паддинг
    )

    # Создаем labels - сдвинутые input_ids на одну позицию
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/20803 [00:00<?, ? examples/s]

## обучение

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    warmup_steps=100,
    logging_steps=100,
    fp16=True,  # Ускорение на GPU с поддержкой
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

Step,Training Loss
100,1.576400
200,1.374500
300,1.595100
400,1.567100
500,1.544200
600,1.529800
700,1.515900
800,1.509700
900,1.523900
1000,1.470500


TrainOutput(global_step=2600, training_loss=1.481497532771184, metrics={'train_runtime': 3526.1758, 'train_samples_per_second': 5.9, 'train_steps_per_second': 0.737, 'total_flos': 1.93169744461824e+16, 'train_loss': 1.481497532771184, 'epoch': 0.9998077292828302})

## инференс

In [ ]:
def generate_title(news_text: str) -> str:
    prompt = f"[TEXT]{news_text}[TITLE]"  # промпт

    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(DEVICE)

    # Параметры генерации
    output = model.generate(
        input_ids,
        max_new_tokens=25,
        temperature=1.1,
        top_p=0.8,
        top_k=0,
        repetition_penalty=1.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        num_beams=1,
    )

    # Декодируем и обрезаем промпт
    full_text = tokenizer.decode(output[0], skip_special_tokens=False)
    title = full_text.split("[TITLE]")[-1].strip()  # Вырезаем заголовок

    return title

In [ ]:
samples = df.sample(5)

inference = []
for index, row in samples.iterrows():
    original_title = row['title']
    text = row['text']
    generated_title = generate_title(text)
    inference.append({'original_title': original_title, 'text': text, 'generated_title': generated_title})

for item in inference:
    print(f"Заголовок:\n\t\t{item['original_title']}")
    print(f"Сгенерированный:\n\t\t{item['generated_title']}")
    print(f"Текст:\n\t\t{item['text']}\n\n")


Заголовок:
		Миссия из Италии, Германии и Греции пролетит над территорией России
Сгенерированный:
		Румынский самолет наблюдения Ан-30 совершил первый испытательный полет в 2019 году. В декабре 2019 года сообщалось, что Румы
Текст:
		 Совместная миссия  Италии ,  Германии  и  Греции  выполнит наблюдательный полет над территорией  России  на румынском самолете  Ан-30  с аэродрома Кубинка, сообщил начальник Национального центра по уменьшению ядерной опасности  Сергей Рыжков . "В период с 9 по 13 марта в рамках реализации Международного договора по открытому небу совместная миссия Италии, Германии и Греции выполнит наблюдательный полет над территорией России на румынском самолете наблюдения Ан-030 с аэродрома Кубинка", — говорится в сообщении. Отмечается, что в ходе полета по согласованному маршруту российские специалисты на борту самолета будут контролировать строгое соблюдение согласованных параметров полета и применение предусмотренной договором аппаратуры наблюдения. Кроме того, Росси

## сохранение чекпоинта

In [ ]:
!zip -r finetuned.zip finetuned

  adding: finetuned/ (stored 0%)
  adding: finetuned/checkpoint-2600/ (stored 0%)
  adding: finetuned/checkpoint-2600/model.safetensors (deflated 7%)
  adding: finetuned/checkpoint-2600/scheduler.pt (deflated 56%)
  adding: finetuned/checkpoint-2600/trainer_state.json (deflated 75%)
  adding: finetuned/checkpoint-2600/training_args.bin (deflated 52%)
  adding: finetuned/checkpoint-2600/generation_config.json (deflated 28%)
  adding: finetuned/checkpoint-2600/config.json (deflated 52%)
  adding: finetuned/checkpoint-2600/optimizer.pt (deflated 9%)
  adding: finetuned/checkpoint-2600/rng_state.pth (deflated 25%)
  adding: finetuned/runs/ (stored 0%)
  adding: finetuned/runs/Feb23_19-13-15_788be442d7ed/ (stored 0%)
  adding: finetuned/runs/Feb23_19-13-15_788be442d7ed/events.out.tfevents.1740337999.788be442d7ed.655.0 (deflated 62%)
